In [ ]:
import pandas as pd
results_df=pd.read_csv("clustering_results.csv")

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import panel as pn

pn.extension("plotly")






# ----------------------------
# Define radar groups
# ----------------------------
radar_groups = {
  "Annotation Agreement Metrics (Max = Best)": ["ARI", "AMI", "Homogeneity"],
  "Annotation Fidelity Metrics (Max = Best)": ["Purity", "Completeness", "V-Measure"],
  "Annotation-Free Metrics (Max = Best)": ["Silhouette-Spatial", "Silhouette", "ASW"],
  "Annotation-Free Metrics (Min = Best)": ["Davies-Bouldin", "CHAOS", "PAS"]
}
# ----------------------------
# Widgets
# ----------------------------
model_selector = pn.widgets.MultiChoice(
    name="Select Methods", 
    value=results_df["method"].tolist(),
    options=results_df["method"].tolist()
)

radar_group_selector = pn.widgets.Select(
    name="Metric Group", 
    options=list(radar_groups.keys()), 
    value="Annotation Agreement Metrics (Max = Best)"  # Default
)

bubble_x = pn.widgets.Select(name="Bubble X Metric", options=list(results_df.columns), value="Silhouette-Spatial")
bubble_y = pn.widgets.Select(name="Bubble Y Metric", options=list(results_df.columns), value="Silhouette")

# ----------------------------
# Create FigureWidget for radar chart
# ----------------------------
fig_radar = go.FigureWidget()

def update_radar(event=None):
    group_name = radar_group_selector.value
    selected_methods = model_selector.value
    metrics = radar_groups[group_name]
    fig_radar.data = []  # Clear existing traces
    for method in selected_methods:
        values = results_df.loc[results_df["method"] == method, metrics].values.flatten().tolist()
        fig_radar.add_trace(go.Scatterpolar(
            r=values,
            theta=metrics,
            fill='toself',
            name=method
        ))
    fig_radar.update_layout( width=800,
        height=800,
        polar=dict(radialaxis=dict(visible=True)),
        showlegend=True,
        title=group_name
    )

# Initial radar chart
update_radar()

# Watch for widget changes
radar_group_selector.param.watch(update_radar, 'value')
model_selector.param.watch(update_radar, 'value')

# ----------------------------
# Bubble chart and bar chart
# ----------------------------
def make_bubble(selected, x_metric, y_metric):
    df_filtered = results_df[results_df["method"].isin(selected)]
    fig = px.scatter(df_filtered, x=x_metric, y=y_metric,
                     size="exec_time", color="method",
                     hover_data=["ARI","AMI","Homogeneity","Purity", 
                            "Completeness","V-Measure","Silhouette-Spatial", "Silhouette", "Davies-Bouldin", "CHAOS", "PAS","ASW" ])
    fig.update_layout(width=800, height=600,title=f"Trade-off: {x_metric} vs {y_metric} (bubble size = exec_time (s))")


    fig.update_layout(
        legend_title_text="Method",
        margin=dict(l=40, r=40, t=40, b=40)
    )
    return fig  



def make_bar(selected):
    df_filtered = results_df[results_df["method"].isin(selected)]
    fig = px.bar(df_filtered, x="method", y=["exec_time","peak_memory"],
                 barmode="group", title="Execution Time (in seconds) & Peak Memory (in MB)")
    fig.update_layout(width=800, height=600)
    return fig

bubble_panel = pn.bind(make_bubble, model_selector, bubble_x, bubble_y)
bar_panel = pn.bind(make_bar, model_selector)



# ----------------------------
# Export helper (save PNG at 300 DPI)
# ----------------------------
def download_png(fig, filename="plot.png"):
    fig.write_image(filename, scale=3)  # 3x scale ~ 300 DPI
    return f"Saved: {filename}"




# ----------------------------
# Biological interpretation (static part)
# ----------------------------
metric_interpretations = {
    "ARI": "Captures faithful domain recovery by measuring agreement between predicted clusters and known tissue domains.",
    "AMI": "Captures the amount of shared information between predicted and reference tissue domains.",
    "Homogeneity": "Relevant for detecting overmixing and consistent reference tissue domain labeling.",
    "Completeness": "Prevents splitting of the same tissue domain.",
    "V-Measure": "Balances avoiding overmixing and undersplitting of tissue domains.",
    "Purity": "Reflects the extent to which each predicted cluster contains spots from only one reference tissue domain.",
    "Silhouette-Spatial": "Relevant for tissues where local microarchitecture matters by balancing spatial adjacency with expression similarity.",
    "Silhouette": "Valuable for datasets where transcriptomic distinction dominates.",
    "Davies-Bouldin": "Relevant for molecularly distinct regions, capturing their low intra-cluster variance and high inter-cluster variance.",
    "CHAOS": "Captures gradual tissue transitions and anatomical boundaries of smooth, contiguous spatial domains.",
    "PAS": "Captures interactions between adjacent cell populations.",
    "ASW": "Beneficial for tissue domains with topologically distinct regions that optimize spatial local density."
}

# ----------------------------
# Function: Build dynamic interpretation table
# ----------------------------
def make_interpretation_table(selected_methods):
    rows = []
    df_filtered = results_df[results_df["method"].isin(selected_methods)]
    for metric, desc in metric_interpretations.items():
        # Sort by metric (ascending for DBI, else descending)
        ascending = True if metric == "Davies-Bouldin" else False
        top2 = df_filtered.sort_values(metric, ascending=ascending)["method"].head(2).tolist()
        rows.append([metric, ", ".join(top2), desc])
    interp_df = pd.DataFrame(rows, 
                             columns=["Metric Category", "Top 2 Performing Method", "Biological Relevance / Interpretation"])
    return interp_df

# ----------------------------
# Panel widget: dynamic table
# ----------------------------
interp_panel = pn.bind(
    lambda selected: pn.widgets.Tabulator(
        make_interpretation_table(selected), 
        pagination="remote", 
        page_size=6, 
        sizing_mode="stretch_width",
        selectable=False
    ),
    model_selector
)




# ----------------------------
# Layout
# ----------------------------
dashboard = pn.Column(
    "# 📊 Spatial Clustering Multimetrics Dashboard",
    pn.Row(model_selector, radar_group_selector  ),
    
    pn.panel(fig_radar  , align="center"),#, sizing_mode="stretch_width"),
    pn.Row( bubble_x, bubble_y ),
    pn.Row(bubble_panel,bar_panel , align="center" ),
    interp_panel, 
)

# Show inline
dashboard.show()


Launching server at http://localhost:40983


In [3]:
#download_png(fig_radar, filename="radar-plot.png")
download_png(fig_radar, filename="radar-plot.png")

RuntimeError: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome



ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'plotly_event', 'values': {'type': 'map', 'entries': [['model', {'id': 'dce6bc6c-55a1-463f-b26d-e3681bb5b326'}], ['data', {'type': 'map', 'entries': [['type', 'hover'], ['data', {'type': 'map', 'entries': [['device_state', {'type': 'map', 'entries': [['alt', False], ['ctrl', False], ['meta', False], ['shift', False], ['button', 0], ['buttons', 0]]}], ['selector', None], ['points', [{'type': 'map', 'entries': [['curveNumber', 9]]}]]]}]]}]]}}}]} 
 error: KeyError('pointNumber')
Traceback (most recent call last):
  File "/home/accounts/personale/nndgpl46/miniconda3/envs/bertwalk2/lib/python3.10/site-packages/bokeh/server/protocol_handler.py", line 94, in handle
    work = await handler(message, connection)
  File "/home/accounts/personale/nndgpl46/miniconda3/envs/bertwalk2/lib/python3.10/site-

In [3]:
!pip install --upgrade kaleido

  Using cached pytest-8.4.2-py3-none-any.whl.metadata (7.7 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
Using cached pytest-8.4.2-py3-none-any.whl (365 kB)
Using cached pluggy-1.6.0-py3-none-any.whl (20 kB)
Using cached iniconfig-2.1.0-py3-none-any.whl (6.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [kaleido]m8/9 [kaleido]imeout]
